## Import

In [1]:
import requests
import os
import pandas as pd
import warnings
import xmltodict
import time
import pickle

from tqdm import tqdm

warnings.filterwarnings('ignore')

* functions

In [ ]:
def extract(select, t_value, y_value, q_value):
    k_value = key_dict[select]
    s_value = service_dict[select]

    start = 1
    end = start + 999

    while True:
        if select != '상권-소득소비':
            base_url = 'http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START}/{END}/'
            URL = base_url.format(KEY=k_value, TYPE = t_value, SERVICE = s_value, START = start, END = end)
        else:
            base_url = 'http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START}/{END}/{STDR_YY_CD}/{STDR_QW_CD}/'
            URL = base_url.format(KEY=k_value, TYPE = t_value, SERVICE = s_value, START = start, END = end, STDR_YY_CD = y_value, STDR_QW_CD = q_value)
            
        r = requests.get(URL)
        res_dict = r.json()

        if start == 1:
            df1 = pd.DataFrame(res_dict[s_value]['row'])
            df1.rename(columns = change_dict, inplace = True)
        else:
            temp_df = pd.DataFrame(res_dict[s_value]['row'])
            temp_df.rename(columns = change_dict, inplace = True)
            df1 = pd.concat([df1, temp_df])

        if df1.shape[0] % 1000 != 0:
            break
        elif 'RESULT' in list(res_dict.keys()):
            break
        else:
            start += 1000
            end += 1000
    return df1

def extract_format(select_list, t_value = 'json', y_value = 2023, q_value = 1):
    start = time.time()
    df_list = []
    
    for select in select_list:
        print(select)
        df1 = extract(select, t_value, y_value, q_value)
        df_list.append(df1)
    
    end = time.time()
    print("수행시간: %f 초" % (end - start))
    
    return df_list

* get data

In [3]:
base_url = 'http://openapi.seoul.go.kr:8088/{KEY}/{TYPE}/{SERVICE}/{START}/{END}/'

service_dict = {'상권영역': 'TbgisTrdarRelm',
             '상권-생활인구': 'VwsmTrdarFlpopQq',
             '상권-직장인구': 'VwsmTrdarWrcPopltnQq',
             '상권-점포': 'VwsmTrdhlStorQq',
             '상권배후지-점포': 'VwsmTrdhlStorQq',
             '상권-추정매출': 'VwsmTrdarSelngQq',
             '상권-상권변화지표': 'VwsmTrdarIxQq',
             '상권-상주인구': 'VwsmTrdarRepopQq',
             '상권-아파트': 'InfoTrdarAptQq',
             '상권-소득소비': 'trdarNcmCnsmp',
             '상권배후지-생활인구': 'VwsmTrdhlFlpopQq',
             '상권배후지-소득소비': 'VwsmTrdhlNcmCnsmpQq',
             '상권배후지-직장인구': 'Vwsm_TrdhlWrcPopltnQq',
             '상권배후지-추정매출': 'VwsmTrdhlSelngQq',
             '상권배후지-아파트': 'VwsmTrdhlAptQq'}

key_dict = {'상권영역':'426279496461757236355952525748',
            '상권-생활인구':'754f44766a617572333456764d687a',
            '상권-직장인구':'4d747762636175723130316f55696246',
            '상권-점포':'624976595761757239316876776a44',
            '상권배후지-점포':'536872566e6175723132354a76466f48',
            '상권-추정매출':'637a59624261757239396e594e7464',
            '상권-상권변화지표':'754563774b61757231323876446b6555',
            '상권-상주인구':'75456564516175723939424656616e',
            '상권-아파트':'6a4f725a4561757231374776577941',
            '상권-소득소비':'455163586461757235306a42657277',
            '상권배후지-생활인구':'656144774b61757232344579745653',
            '상권배후지-소득소비':'70596b475161757235376f616d4c4f',
            '상권배후지-직장인구':'595a67495161757233376f6249745a',
            '상권배후지-추정매출':'6651664c4261757236396f49526c58',
            '상권배후지-아파트':'646e71454461757234316c46495777'}


change_dict = {'TRDAR_SE_CD':'상권_구분_코드',              
                'TRDAR_SE_CD_NM':'상권_구분_코드_명',
                'TRDAR_CD':'상권_코드',
                'TRDAR_CD_NM':'상권_코드_명',
                'XCNTS_VALUE':'엑스좌표_값',
                'YDNTS_VALUE':'와이좌표_값',
                'SIGNGU_CD':'시군구_코드',
                'ADSTRD_CD':'행정동_코드',
                'STDR_YM_CD':'기준_년월_코드'}

select_list = ['상권영역','상권-생활인구','상권-직장인구','상권-점포','상권배후지-점포','상권-추정매출',
               '상권-상권변화지표','상권-상주인구','상권-소득소비', '상권-아파트','상권배후지-생활인구',
               '상권배후지-소득소비','상권배후지-직장인구','상권배후지-추정매출','상권배후지-아파트']

In [6]:
df_test = extract_format(select_list)

상권영역
상권-생활인구
상권-직장인구
상권-점포
상권배후지-점포
상권-추정매출
상권-상권변화지표
상권-상주인구
상권-소득소비
상권-아파트
상권배후지-생활인구
상권배후지-소득소비
상권배후지-직장인구
상권배후지-추정매출
상권배후지-아파트
수행시간: 39124.812449 초


* save dict

In [11]:
data_path = './datasets/'
save_path = data_path + 'market-seoul/'

save_dict = {'name': select_list,
             'data': df_test}

if not os.path.isdir(save_path):
    os.mkdir(save_path)

with open(save_path + 'market-seoul-df.pickle', 'wb') as f:
    pickle.dump(save_dict,f)

## 참고자료

* 상권영역: http://data.seoul.go.kr/dataList/OA-15560/S/1/datasetView.do
* 상권-생활인구: http://data.seoul.go.kr/dataList/OA-15568/S/1/datasetView.do
* 상권-직장인구: http://data.seoul.go.kr/dataList/OA-15569/S/1/datasetView.do
* 상권-점포: https://data.seoul.go.kr/dataList/OA-15577/S/1/datasetView.do
* 상권-추정매출: https://data.seoul.go.kr/dataList/OA-15572/S/1/datasetView.do
* 상권-상권변화지표: https://data.seoul.go.kr/dataList/OA-15576/S/1/datasetView.do
* 상권-상주인구: https://data.seoul.go.kr/dataList/OA-15584/S/1/datasetView.do
* 상권-아파트: https://data.seoul.go.kr/dataList/OA-15566/S/1/datasetView.do
* 상권-소득소비: https://data.seoul.go.kr/dataList/OA-21278/S/1/datasetView.do
* 상권배후지-점포: https://data.seoul.go.kr/dataList/OA-15578/S/1/datasetView.do
* 상권배후지-생활인구: https://data.seoul.go.kr/dataList/OA-15582/S/1/datasetView.do
* 상권배후지-소득소비: https://data.seoul.go.kr/dataList/OA-15571/S/1/datasetView.do
* 상권배후지-직장인구: https://data.seoul.go.kr/dataList/OA-15570/S/1/datasetView.do
* 상권배후지-추정매출: https://data.seoul.go.kr/dataList/OA-15573/S/1/datasetView.do
* 상권배후지-아파트: https://data.seoul.go.kr/dataList/OA-15574/S/1/datasetView.do